### Import and Prep

In [83]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


#import csv
X_output = pd.read_csv("test.csv")
y_output = pd.read_csv("sample_submission.csv")

data_train = pd.read_csv("train.csv")
X = data_train.drop(columns=["Next Week's Deaths"])
y = data_train["Next Week's Deaths"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

#prepare for fit and predict (transform strings to indicators -int-)
Location = data_train["Location"]
le = preprocessing.LabelEncoder()
le.fit(Location)

LocationToInt = le.transform(Location)
LocationDict = dict(zip(Location, LocationToInt))

X_train["Location"] = X_train["Location"].replace(LocationDict)
X_test["Location"] = X_train["Location"].replace(LocationDict)
X_output["Location"] = X_output["Location"].replace(LocationDict)

#cleanse data??
X_train = X_train.drop(["Id"], axis=1)
X_test = X_test.drop(["Id"], axis=1)
X_output = X_output.drop(["Id"], axis=1)


X_train = X_train.fillna(0)
X_test = X_test.fillna(0)
X_output = X_output.fillna(0)
# X_train.head()

### Train and Test

In [84]:
import time


#train run and test
time_start = time.perf_counter()
#model = HistGradientBoostingRegressor(max_bins=255, max_iter=999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999)
model = GradientBoostingRegressor(max_depth=6, n_estimators=2000, subsample=1)
    #early stopping = False possible (really long times)
model.fit(X_train, y_train)
time_elapsed = (time.perf_counter() - time_start)
print(time_elapsed)
print(model.score(X_test,y_test))

#time 1111.4497s = 18,51 min for est=2000 max depth=6 -- score 0.996
#time 72911.8505s = 20,25h for est=150000 max depth=10 -- score 0.994 !!! oof

72911.8505499
0.9940716176210604


### Import model and Reuse

In [ ]:
import pickle

pickled_model = pickle.load(open("model.pkl", "rb"))
print(pickled_model.score(X_test, y_test))

### Dump and Export

In [86]:
import pickle
#dump model
pickle.dump(model, open("model.pkl", "wb"))


#dump output
y_output["Next Week's Deaths"] = model.predict(X_output)
y_output.to_csv("G:\Dateien\-Coden-\Python\Jupyter - Corona Death Prediction\outputlongtraining.csv", index=False)
y_output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43052 entries, 0 to 43051
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  43052 non-null  int64  
 1   Next Week's Deaths  43052 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 672.8 KB
